In [16]:
import pandas as pd
import numpy as np

import nltk
import string
import re
import json

In [17]:
df = pd.read_csv("../xtra/180725C_IR.csv")
df["year"] = df["year"].apply(int)
df["year"] = df["year"].apply(str)

In [18]:
translator = str.maketrans('', '', string.punctuation)
def preprocess_new_line(text):
    text = "".join(text.splitlines())
    text =  text.translate(translator)
    return  " ".join(text.split())

In [19]:
cols = ['lyricist', 'movie', 'song_name', 'year',
       'lyrics_tamil', 'music_director','lines', 'meanings', 'tags',
       'feels']
# for col in cols:
#     df[col] = df[col].apply(preprocess_new_line)
df["lyrics_tamil"] = df["lyrics_tamil"].apply(preprocess_new_line)

In [20]:
json_data = {
    "version": "v3",
    "data" : []
}
for idx,row in df.iterrows():
    json_data["data"].append({
        "திரைப்படம்" : row["movie"],
        "பாடலாசிரியர்" : row["lyricist"],
        "இசையமைப்பாளர்" : row["music_director"],
        "பாடல்" : row["song_name"],
        "வருடம்" : row["year"],
        "பாடல்வரிகள்" : row["lyrics_tamil"],
        "உருவகம்" : row["lines"],
        "விளக்கம்": row["meanings"],
        "உணர்வு" : row["feels"],
        "குறிச்சொல்": row["tags"],
    })

In [24]:
with open("../data/preprocessed_v1_725c.json", 'w', encoding='utf-8-sig') as fp:
    json.dump(json_data, fp, ensure_ascii=False)

In [23]:
json_data

{'version': 'v3',
 'data': [{'திரைப்படம்': 'டிஷ்யூம்',
   'பாடலாசிரியர்': 'வைரமுத்து',
   'இசையமைப்பாளர்': 'Vijay Antony',
   'பாடல்': 'டைலாமோ டைலாமோ',
   'வருடம்': '2006',
   'பாடல்வரிகள்': 'டைலமோ டைலமோ தைல தைல டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ தைல தைல டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ காலைலேகி ரத்ரிமேல் காதலேன் காலைலேகி ரத்ரிமேல் காதலேன் தீக்குச்சிகு தண்ணி மேலே காதலேன் சோ என்பாஸ்கர் என்னோடே லவர் கபார் சோ என்பாஸ்கர் என்னோடே லவர் கபார் டைலமோ டைலமோ தைல தைல டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ தைல தைல டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ அமெரிக்க நீயான ஆப்கானிஸ்தான் நானான கல்யாணம் எந்த ஊரு கூறு உன் ஊரு ஓகேதான் எங்க ஊரும் ஓகேதான் பின் லேடன் என்ன சொல்றான் பாரு உன் மேலே பிட்டு பிட்டு நோய் தீர முட்டு முட்டு உன் மேலே பிட்டு பிட்டு நோய் தீர முட்டு முட்டு காதல் ல சீன் போடன் உம்ஜெட்டு வேகத்தே லா டைலமோ டைலமோ தைல தைல டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ தைல தைல டைலமோ டைலமோ டைலமோ டைலமோ டைலமோ ஸ்ரீ லங்கா நீ ஆனா LTTE நானான ஐய்யோயோ வையை கொஞ்ச மூடு DMK நீ ஆனா ADMK நானான கோட்டைல நம்ம வீட்ட போடு வா